In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import math
import seaborn as sns

%matplotlib inline
tqdm.pandas()

# Prepare dataset

- `bg20_adi`: Neighborhood Atlas's newly published 2020 ADI

In [2]:
bg20_adi = pd.read_csv('../data/US_2020_ADI_Census Block Group_v3.2.csv', 
                        dtype={'FIPS':str, 'ADI_NATRANK':str, 'ADI_STATERNK':str})
bg20_adi = bg20_adi[['FIPS', 'ADI_NATRANK', 'ADI_STATERNK']]
bg20_adi = bg20_adi.rename(columns={'FIPS':'bg_fips'})

In [3]:
bg20_adi

,bg_fips,ADI_NATRANK,ADI_STATERNK
0,010010201001,73,5
1,010010201002,62,3
2,010010202001,83,7
3,010010202002,87,7
4,010010203001,73,5
...,...,...,...
242330,721537506011,92,6
242331,721537506012,87,4
242332,721537506013,93,7
242333,721537506021,98,10


THIS IS THE DATASET WE CREATED.
- `food_valid`: all pairs of food pantry address and block groups within 25 miles of the food pantry, along with longitude/latitude, FIPS, and and geo-distance

Later, we will to merge it with the newly published 2020 ADI values for the updated database.

In [4]:
food_valid = pd.read_pickle('../data/fp_bg_pairs.pkl')

In [5]:
food_valid.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37154567 entries, 0 to 37154566
Data columns (total 8 columns):
 #   Column       Dtype  
---  ------       -----  
 0   address      object 
 1   ad_lat       float64
 2   ad_lon       float64
 3   ad_bg_fips   object 
 4   bg_fips      object 
 5   bg_lat       float64
 6   bg_lon       float64
 7   distance_mi  float64
dtypes: float64(5), object(3)
memory usage: 2.2+ GB


# Data Statistics

- Number of food pantries and banks (FPs) in our dataset: 34475
- Number of unique BGs in our dataset: 232,735

In [6]:
num_unique_pantries = len(food_valid['address'].unique())
print(f"number of FPs in total: {num_unique_pantries}")

num_unique_bgs = len(food_valid['bg_fips'].unique())
print(f"number of unique BGs: {num_unique_bgs}")

number of FPs in total: 34475
number of unique BGs: 238536
